In [1]:
import syft as sy
import sympc
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor
from sympc.protocol import Falcon,FSS
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from typing import Any, Dict, List
import copy
import random
import time
import numpy as np
from tqdm import tqdm
import gc

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

batch_size =200
epochs = 55
train_test_split= 0.8
lr = 0.001
n_client = 70
chosen_prob = 0.6
local_batch_size = 32
local_epochs = 10


class Block(nn.Module):
  def __init__(self, inchannel, outchannel, res=True,stride=0):
    super(Block, self).__init__()
    self.res = res     # With or without residual connection
    self.left = nn.Sequential(
        nn.Conv2d(inchannel, outchannel, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(outchannel),
        nn.ReLU(inplace=True),
        nn.Conv2d(outchannel, outchannel, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(outchannel),
    )
    if stride != 1 or inchannel != outchannel:
        self.shortcut = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=1, bias=False),
            nn.BatchNorm2d(outchannel),
        )
    else:
        self.shortcut = nn.Sequential()

    self.relu = nn.Sequential(
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    out = self.left(x)
    if self.res:
        out += self.shortcut(x)
    out = self.relu(out)
    return out


class Resnet(nn.Module):
  def __init__(self, cfg=[64, 'M', 128,  'M', 256, 'M', 512, 'M'], res=True):
      super(Resnet, self).__init__()
      self.res = res       # With or without residual connection
      self.cfg = cfg       # Configuration list
      self.inchannel = 3   # Number of initial input channels
      self.futures = self.make_layer()
      # The full connection layer and classifier after the construction of the convolution layer:
      self.classifier = nn.Sequential(nn.Dropout(0.4),            
          nn.Linear(4 * 512, 10), )   # fc

  def make_layer(self):
    layers = []
    for v in self.cfg:
      if v == 'M':
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
      else:
        layers.append(Block(self.inchannel, v, self.res))
        self.inchannel = v    # Change the number of input channels to the number of output channels of the previous layer
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.futures(x)
    # view(out.size(0), -1): change tensor size from (N ,H , W) to (N, H*W)
    out = out.view(out.size(0), -1)
    out = self.classifier(out)
    return out


class ResNet18(sy.Module):
  def __init__(self, torch_ref):
    super(ResNet18, self).__init__(torch_ref=torch_ref)
    self.model=Resnet()
  def forward(self, x):
    x = self.model.forward(x.reconstruct().cuda())
    return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465), (0.2023,0.1994,0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='cifar_data', train=True,
                  download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='cifar_data',
                  train=False,
                  download=True,
                  transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,
                     batch_size=batch_size,
                     shuffle=True)
testloader = torch.utils.data.DataLoader(testset,
                    batch_size=batch_size,
                    shuffle=True)



# Get the specified number of clients
def get_clients(n_clients):
  clients = []
  for index in range(n_clients):
      clients.append(sy.VirtualMachine(name="worker" + str(index)).get_root_client())
  return clients

# Divide the data set according to the number of clients
def split_send(data, session):
  data_pointers = []
  split_size = int(len(data) / len(session.parties)) + 1
  for index in range(0, len(session.parties)):
      ptr = data[index * split_size:index * split_size + split_size].share(session=session)
      data_pointers.append(ptr)

  return data_pointers

def train_model(n_clients,dataloader, protocol=None):

  # Get client
  parties = get_clients(n_clients)
  print(parties)
  # Initialize the connection pool according to the number of clients
  # if(protocol):
  #   session = Session(parties=parties, protocol=protocol)
  # else:
  #   session = Session(parties=parties)
  # SessionManager.setup_mpc(session)
  for epoch in tqdm(range(epochs)):
    running_loss = 0.0
    total_correct = 0
    if(protocol):
      session = Session(parties=parties, protocol=protocol)
    else:
      session = Session(parties=parties)
    SessionManager.setup_mpc(session)
    optimizer.zero_grad()
    count=0
    for index, (feature, label) in enumerate(dataloader):
      
      # Split data and send to session
      pointers = split_send(feature, session)
      # Model encryption
      mpc_model = model.share(session)
      results = []
      # The data set is encrypted and calculated using MPC
      for ptr in pointers:
        smpc_results = mpc_model(ptr)
        results.append(smpc_results)
      predictions = torch.cat(results)
      # Loss caculation
      loss = criterion(predictions, label.cuda())
      loss.backward()
      optimizer.step()
      running_loss += np.round(loss.item(),4)
      y_pred_decode = torch.argmax(predictions, dim=1)
      total_correct += y_pred_decode.eq(label.cuda()).sum().item()
      if index >=100:
        break
      
    acc = total_correct/((index+1)*batch_size)
    print(f"Epoch {epoch}/{epochs}  Loss:{running_loss/batch_size} ,accuracy:{acc}")
    torch.cuda.empty_cache()
    test_feature=None
    test_label=None
    
    
def test_model(n_clients,dataloader, protocol=None):
  parties = get_clients(n_clients)
  print(parties)
  for epoch in tqdm(range(1)):
    total_correct = 0
    if(protocol):
      session = Session(parties=parties, protocol=protocol)
    else:
      session = Session(parties=parties)
    SessionManager.setup_mpc(session)
    optimizer.zero_grad()
    for _, (feature, label) in enumerate(dataloader):
      
      # Split data and send to session
      pointers = split_send(feature, session)
      # Model encryption
      mpc_model = model.share(session)
      results = []
      # The data set is encrypted and calculated using MPC
      for ptr in pointers:
        smpc_results = mpc_model(ptr)
        results.append(smpc_results)
      predictions = torch.cat(results)
      y_pred_decode = torch.argmax(predictions, dim=1)
      total_correct += y_pred_decode.eq(label.cuda()).sum().item()
    acc = total_correct/10000
    print(f"test accuracy:{acc}")
    torch.cuda.empty_cache()
    test_feature=None
    test_label=None

model = ResNet18(torch)
model.model.to(device)
sympc.module.SKIP_LAYERS_NAME={"Flatten","Resnet","Small_Model"}
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
print('------------------train------------------------')
train_model(3,trainloader,Falcon("semi-honest"))

------------------train------------------------
[<VirtualMachineClient: worker0 Client>, <VirtualMachineClient: worker1 Client>, <VirtualMachineClient: worker2 Client>]


  2%|▏         | 1/55 [01:40<1:30:38, 100.71s/it]

Epoch 0/55  Loss:1.0267425000000001 ,accuracy:0.3613861386138614


  4%|▎         | 2/55 [03:22<1:29:26, 101.25s/it]

Epoch 1/55  Loss:0.9341705000000001 ,accuracy:0.4226237623762376


...

Epoch 52/55  Loss:0.0614275 ,accuracy:0.958069306930693


 98%|█████████▊| 54/55 [1:32:25<01:44, 104.64s/it]

Epoch 53/55  Loss:0.04560200000000001 ,accuracy:0.9673762376237623


100%|██████████| 55/55 [1:34:11<00:00, 102.75s/it]

Epoch 54/55  Loss:0.0407055 ,accuracy:0.972029702970297


In [4]:
print('------------------test------------------------')
test_model(3,testloader,Falcon("semi-honest"))

------------------test------------------------
[<VirtualMachineClient: worker0 Client>, <VirtualMachineClient: worker1 Client>, <VirtualMachineClient: worker2 Client>]


100%|██████████| 1/1 [00:52<00:00, 52.03s/it]

test accuracy:0.8128
